# `tf.data` API

* References
  * [`tf.data.Dataset` API](https://www.tensorflow.org/api_docs/python/tf/data/Dataset)
  * [Importing data](https://www.tensorflow.org/guide/datasets) (Guideline Documents)
  * [Data Input Pipeline Performance](https://www.tensorflow.org/guide/performance/datasets)

`tf.data` API has two (or three) new abstractions
* `tf.data.Dataset` represents a sequence of elements, in which each element contains one or more Tensor objects. For example, in an image pipeline, the image data and a label
  * Creating a source (e.g. Dataset.from_tensor_slices()) constructs a dataset from one or more `tf.Tensor` objects.
    * `tf.data.Dataset.from_tensors()`
    * `tf.data.Dataset.from_tensor_slices()`
    * `tf.data.TFRecordDataset`:  TFRecord format을 읽을 때
  * Applying a transformation (e.g. Dataset.batch()) constructs a dataset from one or more `tf.data.Dataset` objects.
* `tf.data.Iterator` provides the main way to extract elements from a dataset. The operation returned by `Iterator.get_next()` yields the next element of a `Dataset` when executed, and typically acts as the interface between input pipeline code and your model.

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow.keras import layers
tf.enable_eager_execution()

### Loading MNIST dataset from `tf.keras.datasets`

In [ ]:
# Load training and eval data from tf.keras
(train_data, train_labels), (test_data, test_labels) = \
    tf.keras.datasets.mnist.load_data()

### Show the MNIST

In [ ]:
index = 10
print("label = {}".format(train_labels[index]))
plt.imshow(train_data[index].reshape(28, 28))
plt.colorbar()
#plt.gca().grid(False)
plt.show()

In [ ]:
# set N=50 for small dataset loading
N = 50
train_data = train_data[:N]
train_labels = train_labels[:N]
train_data = train_data / 255.
train_data = train_data.astype(np.float32)
train_labels = train_labels.astype(np.int32)

test_data = test_data[:N]
test_labels = test_labels[:N]
test_data = test_data / 255.
test_data = test_data.astype(np.float32)
test_labels = test_labels.astype(np.int32)

## Input pipeline

1. You must define a source. `tf.data.Dataset`.
  * To construct a Dataset from some tensors in memory, you can use `tf.data.Dataset.from_tensors()` or `tf.data.Dataset.from_tensor_slices()`.
  * Other methods
    * `tf.data.TextLineDataset(filenames)`
    * `tf.data.FixedLengthRecordDataset(filenames)`
    * `tf.data.TFRecordDataset(filenames)`
2. Transformation
  * `Dataset.map()`: to apply a function to each element
  * `Dataset.batch()`
  * `Dataset.shuffle()`
3. `Iterator`
  * `Iterator.initializer`: which enables you to (re)initialize the iterator's state
  * `Iterator.get_next()`

### 1. Store data in `tf.data.Dataset`

* `tf.data.Dataset.from_tensor_slices((features, labels))`
* `tf.data.Dataset.from_generator(gen, output_types, output_shapes)`

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_data, train_labels))
print(train_dataset)
print(train_dataset.output_shapes)
print(train_dataset.output_types)

### 2. Transformaion

* `apply(transformation_func)`
* `batch(batch_size)`
* `concatenate(dataset)`
* `map(map_func)`
* `repeat(count=None)`
  * count=max_epochs
* `shuffle(buffer_size, seed=None, reshuffle_each_iteration=None)`

In [ ]:
def cast_label(image, label):
  label = tf.cast(label, dtype=tf.float32)
  return image, label

In [ ]:
batch_size = 16

train_dataset = tf.data.Dataset.from_tensor_slices((train_data, train_labels))
train_dataset = train_dataset.shuffle(buffer_size = 10000)
train_dataset = train_dataset.repeat(count=2)
train_dataset = train_dataset.map(cast_label)
train_dataset = train_dataset.batch(batch_size = batch_size)

### 3. Iterator

* In eager execution, we can directly sample the batch dataset from `tf.data.Dataset` instead of using `Iterator`.
* We do not need to explicitly create an `tf.data.Iterator` object. 

In [ ]:
for step, (x, y) in enumerate(train_dataset.take(10)):
  print("step: {}  label: {}".format(step, y))

#### General method for importing data

In [ ]:
batch_size = 16

train_dataset = tf.data.Dataset.from_tensor_slices((train_data, train_labels))
train_dataset = train_dataset.shuffle(buffer_size = 10000)
train_dataset = train_dataset.batch(batch_size = batch_size, drop_remainder=True)

In [ ]:
max_epochs = 2
for epoch in range(max_epochs):
  for step, (x, y) in enumerate(train_dataset):
    print("step: {}  label: {}".format(step, y))